In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

torch.manual_seed(0)
X = torch.rand(1000, 10)
y = (torch.sum(X, dim=1) > 5).float().unsqueeze(1)

train_data = TensorDataset(X[:800], y[:800])
val_data = TensorDataset(X[800:], y[800:])
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)


class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)


def generate_adversarial_examples(model, inputs, labels, epsilon=0.1):
    inputs.requires_grad = True
    outputs = model(inputs)
    loss = nn.BCELoss()(outputs, labels)
    loss.backward()
    adversarial_inputs = inputs + epsilon * inputs.grad.sign()
    return torch.clamp(adversarial_inputs, 0, 1)

def tangent_vector(inputs, transformation="rotation", angle=0.1):
    if transformation == "rotation":
        return inputs + angle * torch.rand_like(inputs)

    return inputs

def tangent_propagation_loss(model, inputs):
    perturbed_inputs = tangent_vector(inputs)
    original_outputs = model(inputs)
    perturbed_outputs = model(perturbed_inputs)
    return nn.MSELoss()(original_outputs, perturbed_outputs)

model = NeuralNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

def train_adversarial_tangent(model, train_loader, val_loader, num_epochs=10, epsilon=0.1, lambda_tangent=0.1):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:

            outputs = model(inputs)
            loss_clean = criterion(outputs, labels)


            adversarial_inputs = generate_adversarial_examples(model, inputs, labels)
            outputs_adv = model(adversarial_inputs)
            loss_adversarial = criterion(outputs_adv, labels)


            loss_tangent = tangent_propagation_loss(model, inputs)


            loss = loss_clean + loss_adversarial + lambda_tangent * loss_tangent


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()


        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()


        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


train_adversarial_tangent(model, train_loader, val_loader)
